# **How trustworthy is your local representative?**

#### Data Sources:
politifact api documentation: http://static.politifact.com/api/v2apidoc.html <br>
govtrack api documentation: https://www.govtrack.us/developers/api <br>
propublica: https://projects.propublica.org/api-docs/congress-api/ <br>
congressional misconduct database: https://www.govtrack.us/misconduct <br>
news api: https://newsapi.org/docs <br>
votesmart: https://github.com/votesmart/python-votesmart/tree/master

## To do: April 15

1. Scrape truth-o-meter data ✅
2. Join truth-o-meter, misconduct db, and propublica ✅
3. Structure bills & votes **(in-progress)**
4. Incorporate news headlines
5. incorprate tweets (?)


In [1]:
# install Propublica congress api client module (from https://github.com/eyeseast/propublica-congress)
# ! pip install python-congress # <= uncomment this here

In [2]:
import requests

import json

import pandas as pd
import numpy as np
import datetime as dt

from congress import Congress

from IPython.display import clear_output

import os
import time
import ast

import requests
from lxml import html

## Get Politifact Data
politifact api documentation: http://static.politifact.com/api/v2apidoc.html 

In [3]:
# Some Functions

def get_politifact(limit,offset):
    '''Scrapes data from politifact API'''
    pf_url = 'http://www.politifact.com/api/v/2/statement/'

    params = {
        'format' : 'json',
        'edition__edition_slug':'truth-o-meter',
        'limit' : limit,
        'offset' : offset,
        'order_by' : 'ruling_date',
    }

    resp = requests.get(pf_url,params=params).json()
    pf = pd.DataFrame(resp['objects'])
    return pf

def restart_politifact(json_path):
    """Restart Politifact Scrape"""
    df = get_politifact(100,0)
    df.to_json(json_path)
    
def refresh_politifact(json_path):
    """Refresh Politifact CSV"""
    cols = get_politifact(1,0).columns
    df = pd.read_json(json_path)
    print('Pre-refresh length:',len(df))
    df = df.drop_duplicates('statement')[cols]
    print('Post-refreshed length:',len(df))
    df.to_json(json_path)
    
def preview_cols(df):
    for col in df.columns:
        display(df[col].head(3))

### get_politifact.py script (markdown)

Unix commands: <br>
pause: ctr + c <br>
resume: fg <br>
```python
import time
import pandas as pd
import requests
import json
import os

print('\nGet Politifact\n')

limit = 100
file_name = 'Politifact.json'
folder_name = 'Data'

file_name = '/' + file_name
folder_name = '/' + folder_name

json_path = os.getcwd()+folder_name+file_name

def get_politifact(limit,offset):
    """Scrape Politifact API"""
    pf_url = 'http://www.politifact.com/api/v/2/statement/'
    params = {
        'format' : 'json',
        'edition__edition_slug':'truth-o-meter',
        'limit' : limit,
        'offset' : offset,
        'order_by' : 'ruling_date'}
    resp = requests.get(pf_url,params=params).json()
    pf = pd.DataFrame(resp['objects'])
    return pf

if not os.path.isdir(os.getcwd()+folder_name):
    os.mkdir(os.getcwd()+folder_name)
    print('* New dir made: ',os.getcwd()+folder_name)

if not os.path.isfile(json_path):
    t1 = time.time()
    df = get_politifact(100,0)
    df.to_json(json_path)
    print('* New file made:',json_path,'\n')
    print('data_length:',len(df))
    timer = time.time() - t1
    print('runtime: {rt} seconds'.format(rt=round(timer,2)))
    time.sleep(5)
print('*'*20)

run = True
while run:
    try:
        t1 = time.time()
        old_df = pd.read_json(json_path)
        old_offset = len(old_df)+1

        print('data_length:',len(old_df))
        print('Offset:',old_offset)
        print('Getting data...')

        new_df = get_politifact(limit,old_offset)
        cols = new_df.columns

        df = old_df.append(new_df).drop_duplicates('statement').reset_index()[cols]
        df.to_json(json_path)
        
        print('---')
        print('data_length:',len(df))
        
        exp = old_offset+limit
        print('Expected offset:',exp)
        print('---')

        new_offset = len(df)+1
        print('New offset:',new_offset)

        timer = time.time() - t1
        print('runtime: {rt} seconds'.format(rt=round(timer,2)))

        if exp < new_offset:
            print('#'*10,'OFFSET ERROR','#'*10)
        elif new_offset > exp:
            print('OFFSET Diff:',new_offset-exp)

        if new_offset == old_offset:
            print('#'*10,'OFFSET STABLE','#'*10)
            break

        time.sleep(5)
    except:
        print('#'*10,'EXCEPT ACTIVATED','#'*10)
        run = False
    print('='*20)
```

# Politifact

In [4]:
json_path = os.getcwd()+'/Data/Politifact.json'
pf = pd.read_json(json_path)
pf.sort_values('statement_date',ascending=False,inplace=True)
pf.reset_index(inplace=True)

pf['statement_date'] = pd.to_datetime(pf['statement_date'])
pf['ruling_date'] = pd.to_datetime(pf['ruling_date'])
pf['statement'] = (pf['statement'].str.replace('<p>',"").str.replace('</p>','')
                   .str.replace('"','').str.replace('&quot;','')
                   .str.replace('\r','').str.replace('\n','').str.replace('&#39;','\''))
pf['ruling_comments'] = (pf['ruling_comments'].str.replace('<p>',"").str.replace('</p>','')
                   .str.replace('"','').str.replace('&quot;','')
                   .str.replace('\r','').str.replace('\n','').str.replace('&#39;','\''))

pf['ruling_slug'] = [ruling['ruling_slug'] for ruling in pf['ruling']]

pf['name_slug'] = [speaker['name_slug'] for speaker in pf['speaker']]
pf['home_state'] = [speaker['home_state'] for speaker in pf['speaker']]
pf['party'] = [speaker['party']['party_slug'] for speaker in pf['speaker']]
pf['first_name'] = [speaker['first_name'] for speaker in pf['speaker']]
pf['last_name'] = [speaker['last_name'] for speaker in pf['speaker']]

pf['statement_type'] = [statement['statement_type'] for statement in pf['statement_type']]

pf['subject_slug'] = [[subject['subject_slug'] for subject in subjects] for subjects in pf['subject']]



In [5]:
pf['speaker'][0]

{'barely_true_count': 1,
 'current_job': 'U.S. Senator',
 'false_count': 0,
 'first_name': 'Tina',
 'half_true_count': 0,
 'home_state': 'Minnesota',
 'id': 4282,
 'last_name': 'Smith',
 'mostly_true_count': 0,
 'name_slug': 'tina-smith',
 'pants_count': 0,
 'party': {'id': 1,
  'party': 'Democrat',
  'party_slug': 'democrat',
  'resource_uri': '/api/v/2/party/1/'},
 'photo': 'http://static.politifact.com.s3.amazonaws.com/politifact/mugs/Tina_Smith_mug.jpg',
 'primary_edition': {'edition': 'National',
  'edition_slug': 'truth-o-meter',
  'id': 1,
  'meter_name': 'The Truth-O-Meter<sup>TM</sup>',
  'resource_uri': '/api/v/2/edition/1/'},
 'promise_meter_cutout': None,
 'resource_uri': '/api/v/2/person/4282/',
 'total_count': 1,
 'true_count': 0,
 'website': 'https://www.smith.senate.gov/'}

In [6]:
# Test to see that the cleaning method worked

sub = 'last_name'
main = 'speaker'
test_num = 10

import random
test = [random.randint(0,len(pf)) for i in range(test_num)]
for n in test:
    print(pf[sub].iloc[n] == pf.iloc[n][main][sub],'\t',pf[sub].iloc[n],pf.iloc[n][main][sub])

True 	 Obama Obama
True 	 Priorities USA Action Priorities USA Action
True 	 Steele Steele
True 	 Hannity Hannity
True 	 Clinton Clinton
True 	 Cruz Cruz
True 	 Pence Pence
True 	 Clinton Clinton
True 	 Rubio Rubio
True 	 McCain McCain


In [7]:
pf.columns

Index(['index', 'art', 'author', 'canonical_url', 'edition', 'editor',
       'facebook_headline', 'id', 'in_future', 'is_pundit', 'make_public',
       'preview', 'researcher', 'resource_uri', 'ruling', 'ruling_comments',
       'ruling_comments_date', 'ruling_date', 'ruling_headline',
       'ruling_headline_slug', 'ruling_link_text', 'ruling_state',
       'source_documents', 'sources', 'speaker', 'statement',
       'statement_context', 'statement_date', 'statement_type', 'subject',
       'target', 'twitter_headline', 'ruling_slug', 'name_slug', 'home_state',
       'party', 'first_name', 'last_name', 'subject_slug'],
      dtype='object')

In [8]:
politifact = pf[['ruling_slug','first_name','last_name','name_slug',
                 'home_state','party','statement',
                 'subject_slug','ruling_comments','ruling_date',
                 'statement_context','statement_type','statement_date',
                 'twitter_headline','sources']]
politifact.head()

,ruling_slug,first_name,last_name,name_slug,home_state,party,statement,subject_slug,ruling_comments,ruling_date,statement_context,statement_type,statement_date,twitter_headline,sources
0,barely-true,Tina,Smith,tina-smith,Minnesota,democrat,"On average, American women only earn 80 cents ...","[diversity, jobs, women]",On April 10 -- Equal Pay Day -- Sen. Tina Smit...,2018-04-13 12:27:18,a tweet,Claim,2018-04-10,Do women get only 80 percent of the pay men do...,"<p>Tina Smith, <a href=""https://twitter.com/Se..."
1,mostly-true,Donald,Trump,donald-trump,New York,republican,When a car is sent to the United States from C...,"[china, trade]",President Donald Trump took to Twitter to bemo...,2018-04-09 17:06:15,in a tweet,Claim,2018-04-09,Does China slap a 25 percent tariff on America...,"<p><a href=""https://twitter.com/realDonaldTrum..."
2,barely-true,Donald,Trump,donald-trump,New York,republican,EPA administrator Scott Pruitt's short-term re...,[ethics],When Environmental Protection Agency administr...,2018-04-10 11:09:40,a tweet,Claim,2018-04-07,Did EPA's Pruitt pay below market rent?,"<p>Donald Trump, <a href=""https://twitter.com/..."
3,barely-true,Donald,Trump,donald-trump,New York,republican,Says Scott Pruitt&rsquo;s security spending wa...,"[environment, federal-budget, homeland-security]",President Donald Trump came out in defense of ...,2018-04-10 11:08:25,a tweet,Claim,2018-04-07,Comparing Scott Pruitt's security spending wit...,"<p>Donald Trump, <a href=""https://twitter.com/..."
4,barely-true,Donald,Trump,donald-trump,New York,republican,This will be the last time &mdash; April &mdas...,[taxes],If you are struggling to finish and file your ...,2018-04-09 08:00:00,an event in West Virginia,Claim,2018-04-05,Can taxpayers expect card-sized tax forms next...,"<p>Donald Trump, <a href=""https://www.whitehou..."


# Govtrack (Deprecated)
govtrack api documentation: https://www.govtrack.us/developers/api 

In [9]:
# Data on legislators

person = 'https://www.govtrack.us/api/v2/person'
role = 'https://www.govtrack.us/api/v2/role'

params = {
    'current' : 'true'
}

resp = requests.get(person).json()

legis = pd.DataFrame(resp['objects'])
hillary = legis[legis['lastname']=='Clinton']['bioguideid'].iloc[0]

# Propublica
propublica: https://projects.propublica.org/api-docs/congress-api/  <br>
api client github: https://github.com/eyeseast/propublica-congress

In [10]:
# activate to install propublica congress library
# ! pip install python-congress

In [11]:
from congress import Congress

propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'
congress = Congress(propublica_key)

cong = 80 # Earliest congress is 80
cong = 115 # Latest is 115

st = congress.members.filter('senate',congress = cong)
hs = congress.members.filter('house', congress = cong)

senate_cols = ['id','lis_id','state','state_rank','party',
               'first_name','last_name','middle_name',
               'gender','date_of_birth','in_office',
               'votes_with_party_pct','missed_votes_pct','missed_votes',
               'next_election','title','seniority', # Seniority = years served
               'leadership_role','dw_nominate', # look into dw_nominate
               'twitter_account']

house_cols = ['id','state','district','party',
              'first_name','last_name','middle_name',
              'gender','date_of_birth','in_office',
              'votes_with_party_pct','missed_votes_pct','missed_votes',
              'next_election','title','seniority',
              'leadership_role','dw_nominate',
              'twitter_account']

senate = pd.DataFrame(st[0]['members'])[senate_cols]
house = pd.DataFrame(hs[0]['members'])[house_cols]

### Creating Foreign Key for Propublica and Politifact

In [12]:
# Connecting Senate to Politifact with name slug
senate['name_slug'] = senate['first_name'].str.lower() + '-' + senate['last_name'].str.lower()

filter1 = senate[~senate['name_slug'].isin(politifact['name_slug'])]
filter2 = filter1[filter1['last_name'].isin(politifact['last_name'])]
sen_not_pf = filter1[~filter1['last_name'].isin(politifact['last_name'])]

new_slugs = list()

for i in range(len(filter2)):
    name = ['first_name','last_name']
    col = 'last_name'
    query = filter2['last_name'].iloc[i]
    result = politifact[politifact[col] == query]
    slug = result['name_slug'].iloc[0]
    
    new_slugs.append(slug)
    '''
    print(filter2[name].iloc[i])
    display(result[name])
    print(slug)
    print('='*30)
    '''
    
new_slugs = [slug if slug != 'joe-kennedy' else 'john-kennedy' for slug in new_slugs]
new_slugs = [slug if slug != 'mark-udall' else 'tom-udall' for slug in new_slugs]

# Change name slugs
filter2.loc[:,'name_slug'] = new_slugs

for index in filter2.index:
    condition = senate.loc[index]['name_slug'] == filter2.loc[index]['name_slug']
    if not condition:
        print('old:',senate.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])
        senate.loc[index,'name_slug'] = filter2.loc[index]['name_slug']
        print('new:',senate.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])
    print('='*20)

old: shelley-capito 	 shelley-moore-capito
new: shelley-moore-capito 	 shelley-moore-capito
old: benjamin-cardin 	 ben-cardin
new: ben-cardin 	 ben-cardin
old: christopher-coons 	 chris-coons
new: chris-coons 	 chris-coons
old: michael-enzi 	 mike-enzi
new: mike-enzi 	 mike-enzi
old: charles-grassley 	 chuck-grassley
new: chuck-grassley 	 chuck-grassley
old: christopher-murphy 	 chris-murphy
new: chris-murphy 	 chris-murphy
old: jim-risch 	 james-risch
new: james-risch 	 james-risch
old: bernard-sanders 	 bernie-s
new: bernie-s 	 bernie-s
old: patrick-toomey 	 pat-toomey
new: pat-toomey 	 pat-toomey
old: chris-van hollen 	 chris-van-hollen
new: chris-van-hollen 	 chris-van-hollen


/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
# # Connecting house to Politifact with name_slug

house['name_slug'] = house['first_name'].str.lower() + '-' + house['last_name'].str.lower()

filter1 = house[~house['name_slug'].isin(politifact['name_slug'])]
filter2 = filter1[filter1['last_name'].isin(politifact['last_name'])]
rep_not_pf = filter1[~filter1['last_name'].isin(politifact['last_name'])]

new_slugs = list()
count = 0
for i in range(len(filter2)):
    name = ['first_name','last_name']
    col = 'last_name'
    
    match = filter2['last_name'].iloc[i]
    
    result = politifact[politifact[col] == match]
    
    nf = 2
    result = result[result['first_name'].str[:nf] == filter2['first_name'].str[:nf].iloc[i]]

    if len(result) > 0:
        slug = result['name_slug'].iloc[0]
        new_slugs.append(slug)
        count += 1
        '''
        print('#new slug appended#')
        print(filter2[name].iloc[i])
        display(result[name])
        print('slug:',slug)
        print('='*30)
        '''
    else:
        slug = filter2['name_slug'].iloc[i]
        new_slugs.append(slug)

# Change name slugs
filter2.loc[:,'name_slug'] = new_slugs
house

for index in filter2.index:
    condition = house.loc[index]['name_slug'] == filter2.loc[index]['name_slug']
    if not condition:
        print('old:',house.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])
        house.loc[index,'name_slug'] = filter2.loc[index]['name_slug']
        print('new:',house.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])
        print('='*20)

old: donald-beyer 	 don-beyer
new: don-beyer 	 don-beyer
old: david-brat 	 dave-brat
new: dave-brat 	 dave-brat
old: andré-carson 	 andre-carson
new: andre-carson 	 andre-carson
old: joaquín-castro 	 joaquin-castro
new: joaquin-castro 	 joaquin-castro
old: gerald-connolly 	 gerry-connolly
new: gerry-connolly 	 gerry-connolly
old: jeffrey-duncan 	 jeff-duncan
new: jeff-duncan 	 jeff-duncan
old: sheila-jackson lee 	 sheila-jackson-lee
new: sheila-jackson-lee 	 sheila-jackson-lee
old: eddie-johnson 	 eddie-bernice-johnson
new: eddie-bernice-johnson 	 eddie-bernice-johnson
old: joseph-kennedy 	 joe-kennedy
new: joe-kennedy 	 joe-kennedy
old: raúl-labrador 	 raul-labrador
new: raul-labrador 	 raul-labrador
old: cathy-mcmorris rodgers 	 cathy-mcmorris-rodgers
new: cathy-mcmorris-rodgers 	 cathy-mcmorris-rodgers
old: beto-o'rourke 	 beto-orourke
new: beto-orourke 	 beto-orourke
old: debbie-wasserman schultz 	 debbie-wasserman-schultz
new: debbie-wasserman-schultz 	 debbie-wasserman-schultz


/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
house[house['name_slug']=='beto-orourke']

,id,state,district,party,first_name,last_name,middle_name,gender,date_of_birth,in_office,votes_with_party_pct,missed_votes_pct,missed_votes,next_election,title,seniority,leadership_role,dw_nominate,twitter_account,name_slug
301,O000170,TX,16,D,Beto,O'Rourke,None,M,1972-09-26,True,92.04,1.98,17.0,2018,Representative,6,None,-0.319,RepBetoORourke,beto-orourke


In [34]:
warren = senate[senate['last_name'] == 'Warren']['id'].iloc[0]

In [35]:
warren

'W000817'

In [87]:
senate.to_json('senate.json')

### Connecting to Congressional Misconduct DB

Use `Xie_PPDS_HW3.ipynb` to scrape data from govtrack misconduct database

In [15]:
from sqlalchemy import create_engine

pw = 'dwdstudent2015'

conn_string = 'mysql://{user}:{password}@{host}:{port}/?charset=utf8'.format(
    user     = 'root', 
    password = input('password'), 
    host     = '127.0.0.01', 
    port     = 3306, 
    encoding = 'utf-8'
)

clear_output()

engine = create_engine(conn_string)

engine.execute('USE congressional_misconduct')

query = 'SELECT * FROM misconduct_entries'
misconduct = pd.read_sql(query,con=engine)

## Upload to SQL

In [37]:
db = 'congress'
engine.execute('CREATE DATABASE IF NOT EXISTS {db}'.format(db=db))

/anaconda/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1007, "Can't create database 'congress'; database exists")
  cursor.execute(statement, parameters)


In [52]:
engine.execute('USE {db}'.format(db=db))
query='SHOW Databases'
display(pd.read_sql(query,con=engine))
query = 'SHOW Tables'
display(pd.read_sql(query,con=engine))

,Database
0,information_schema
1,congress
2,congressional_misconduct
3,mysql
4,performance_schema
5,portfolio_analysis
6,sys


,Tables_in_congress
0,senate_115


In [51]:
drop_table = 'senate'
engine.execute('DROP TABLE IF EXISTS {table}'.format(table = drop_table))

In [94]:
len(senate['id'][0])

7

In [96]:
df = senate
sql_cols = senate.columns
table_name = 'senate_115'
primary_key = 'id(10)'

engine.execute('DROP TABLE IF EXISTS {table}'.format(table = table_name))
df[sql_cols].to_sql(name=table_name,
                    con=engine,
                    if_exists='replace',
                    index=False)

add_key_query = 'ALTER TABLE {table} ADD PRIMARY KEY({key})'.format(table=table_name,key=primary_key)
engine.execute(add_key_query)

In [66]:
cols = ['first_name','last_name','party','state','title','seniority']
senate[cols].tail()

,first_name,last_name,party,state,title,seniority
99,Elizabeth,Warren,D,MA,"Senator, 1st Class",5
100,Sheldon,Whitehouse,D,RI,"Senator, 1st Class",11
101,Roger,Wicker,R,MS,"Senator, 1st Class",11
102,Ron,Wyden,D,OR,"Senator, 3rd Class",22
103,Todd,Young,R,IN,"Senator, 3rd Class",1


In [81]:
select = ['first_name','last_name','party','state','title','seniority','id']
table_name = 'senate_115'
condition = 'WHERE last_name = "Warren"'

query = ('''SELECT {columns} FROM {table} {condition}'''
         .format(columns = ','.join(select),table=table_name,condition = condition))

display(pd.read_sql(query,con=engine))

,first_name,last_name,party,state,title,seniority,id
0,Elizabeth,Warren,D,MA,"Senator, 1st Class",5,W000817


### Merging Politifact, Propublica, and Congressional Misconduct DB

In [16]:
# Merge misconduct with senate
pd.merge(misconduct,senate[senate_cols[:12]],
         how='inner',left_on='bioguide',right_on='id').head()

,entry_index,member_name,bioguide,party_x,district,member,member_page,paragraph,id,lis_id,state,state_rank,party_y,first_name,last_name,middle_name,gender,date_of_birth,in_office,votes_with_party_pct
0,370,Sen. Robert “Bob” Menéndez,M000639,Democrat,[D-NJ],Sen. Robert “Bob” Menéndez [D-NJ],https://www.govtrack.us/congress/members/rober...,Menendez is being investigated for unspecified...,M000639,S306,NJ,senior,D,Robert,Menendez,None,M,1954-01-01,True,92.57
1,373,Sen. Alan “Al” Franken,F000457,Democrat,"[D-MN, 2009-2017]","Sen. Alan “Al” Franken [D-MN, 2009-2017]",https://www.govtrack.us/congress/members/alan_...,"On December 7, 2017, Senator Franken announced...",F000457,S332,MN,junior,D,Al,Franken,None,M,1951-05-21,False,97.44


In [17]:
# Merge misconduct with house
pd.merge(misconduct,house[house_cols[:12]]
         ,how='inner',left_on='bioguide',right_on='id').sort_values('bioguide').head()

,entry_index,member_name,bioguide,party_x,district_x,member,member_page,paragraph,id,state,district_y,party_y,first_name,last_name,middle_name,gender,date_of_birth,in_office,votes_with_party_pct,missed_votes_pct
7,293,Rep. Robert Aderholt,A000055,Republican,[R-AL4],Rep. Robert Aderholt [R-AL4],https://www.govtrack.us/congress/members/rober...,In 2010 Aderholt was investigated for keeping ...,A000055,AL,4,R,Robert,Aderholt,B.,M,1965-07-22,True,96.75,3.38
58,380,Rep. Robert Brady,B001227,Democrat,[D-PA1],Rep. Robert Brady [D-PA1],https://www.govtrack.us/congress/members/rober...,Brady faced an allegation of leading a crimina...,B001227,PA,1,D,Robert,Brady,A.,M,1945-04-07,True,94.34,3.38
52,365,Rep. Madeleine Bordallo,B001245,Democrat,[D-GU0],Rep. Madeleine Bordallo [D-GU0],https://www.govtrack.us/congress/members/madel...,In 2017 Bardallo was investigated for receivin...,B001245,GU,At-Large,D,Madeleine,Bordallo,Z.,F,1933-05-31,True,NaN,NaN
10,297,Rep. George “G.K.” Butterfield,B001251,Democrat,[D-NC1],Rep. George “G.K.” Butterfield [D-NC1],https://www.govtrack.us/congress/members/georg...,In 2010 Butterfield was investigated for keepi...,B001251,NC,1,D,G.,Butterfield,K.,M,1947-04-27,True,94.44,5.82
19,308,Rep. Vern Buchanan,B001260,Republican,[R-FL16],Rep. Vern Buchanan [R-FL16],https://www.govtrack.us/congress/members/vern_...,In 2011 Buchanan was investigated for failing ...,B001260,FL,16,R,Vern,Buchanan,None,M,1951-05-08,True,96.50,6.98


In [18]:
# Merge Senate and Misconduct
df = pd.merge(politifact,senate,
         how='inner',left_on='name_slug',right_on='name_slug')
cols = df.columns[:len(politifact.columns)+10]
df[cols].sort_values('name_slug').head()

,ruling_slug,first_name_x,last_name_x,name_slug,home_state,party_x,statement,subject_slug,ruling_comments,ruling_date,...,id,lis_id,state,state_rank,party_y,first_name_y,last_name_y,middle_name,gender,date_of_birth
315,mostly-true,Al,Franken,al-franken,Minnesota,democrat,President (Ronald) Reagan did a similar thing....,[immigration],"Sen. Al Franken, D-Minn., took to the airwaves...",2017-09-08 15:23:05,...,F000457,S332,MN,junior,D,Al,Franken,None,M,1951-05-21
633,mostly-true,Amy,Klobuchar,amy-klobuchar,Minnesota,democrat,Human trafficking is the third-biggest crimina...,"[crime, human-rights]","Sen. Amy Klobuchar, D-Minn., told delegates at...",2016-07-26 22:45:27,...,K000367,S311,MN,senior,D,Amy,Klobuchar,None,F,1960-05-25
634,true,Amy,Klobuchar,amy-klobuchar,Minnesota,democrat,"<div dir=ltr style=color: rgb(0, 0, 0); font-f...",[energy],Considering the United States&rsquo; strength ...,2015-01-04 18:37:38,...,K000367,S311,MN,senior,D,Amy,Klobuchar,None,F,1960-05-25
635,mostly-true,Amy,Klobuchar,amy-klobuchar,Minnesota,democrat,Children who witness domestic violence are twi...,"[crime, criminal-justice, women]",The release of a video showing NFL running bac...,2014-09-16 11:44:47,...,K000367,S311,MN,senior,D,Amy,Klobuchar,None,F,1960-05-25
78,mostly-true,Angus,King,angus-king,Maine,independent,"China, my understanding is, supplies 4 percent...","[china, foreign-policy, trade]",President Donald Trump said his administration...,2018-03-08 09:30:00,...,K000383,S363,ME,junior,I,Angus,King,None,M,1944-03-31


In [19]:
# Merge House and Misconduct
df = pd.merge(politifact,house,
         how='inner',left_on='name_slug',right_on='name_slug')
cols = df.columns[:len(politifact.columns)+10]
df[cols].sort_values('name_slug').head()

,ruling_slug,first_name_x,last_name_x,name_slug,home_state,party_x,statement,subject_slug,ruling_comments,ruling_date,...,id,state,district,party_y,first_name_y,last_name_y,middle_name,gender,date_of_birth,in_office
179,false,Adam,Kinzinger,adam-kinzinger,Illinois,republican,The current debate over authorizing military a...,"[congress, history, military, terrorism]",Months after the Islamic State emerged as a th...,2015-02-19 14:29:29,...,K000378,IL,16,R,Adam,Kinzinger,None,M,1978-02-27,True
181,half-true,Adam,Kinzinger,adam-kinzinger,Illinois,republican,"Since Debbie Halvorson has been in politics, I...","[economy, message-machine]",With unemployment rates near their post-World ...,2010-10-07 10:07:49,...,K000378,IL,16,R,Adam,Kinzinger,None,M,1978-02-27,True
180,false,Adam,Kinzinger,adam-kinzinger,Illinois,republican,"<div style=color: rgb(0, 0, 0); font-family: a...","[federal-budget, military]",Midterm elections are less than two weeks away...,2014-10-26 17:41:57,...,K000378,IL,16,R,Adam,Kinzinger,None,M,1978-02-27,True
107,true,Adam,Schiff,adam-schiff,California,democrat,I think what the authors of the (25th) Amendme...,"[congress, disability, legal-issues]",A Democratic lawmaker who disapproves of Presi...,2017-08-29 13:09:42,...,S001150,CA,28,D,Adam,Schiff,B.,M,1960-06-22,True
2,mostly-true,Alex,Mooney,alex-mooney,West Virginia,republican,"From 2000-10, U.S. manufacturing employment sh...","[economy, jobs]",There&rsquo;s no question that manufacturing e...,2018-04-04 14:36:24,...,M001195,WV,2,R,Alex,Mooney,None,M,1971-06-05,True


# Getting Bills
https://projects.propublica.org/api-docs/congress-api/bills/

In [20]:
def bill_subject(bill_id):
    
    '''Gets list of bill subjects given bill_id'''
    
    bill_slug, cong = bill_id.split('-')
    bill = pd.DataFrame(congress.bills.subjects(bill_slug,congress=cong))['subjects']
    return [subject['url_name'] for subject in bill]


def add_bill_subjects(bills):
    
    '''add bill subjects to list of bill dictionaries'''
    print('Getting bill subjects...\n')
    n=0
    for bill in bills:
        bill_subjects = bill_subject(bill['bill_id'])
        bill['bill_subjects'] = bill_subjects

        print(len(bills)-n, 'bills left')
        n+=1
        if n%10 == 0:
            clear()
            print('Getting bill subjects...\n')
    clear()
    return bills

def get_bills(bioguide,bill_type = 'introduced',get_subjects=True):
    
    '''get all bills introduced by a member'''
    
    import requests
    import time
    from IPython.display import clear_output as clear
    
    '''Scrapes all bills introduced or otherwise related to a member'''
    
    PROPUBLICA_API_KEY = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'

    propub_url = ('https://api.propublica.org/congress/v1/members/{m_id}/bills/{bill_type}.json'
                  .format(m_id=bioguide,bill_type=bill_type))
    header = {'X-API-Key' : PROPUBLICA_API_KEY}

    page=0
    res = list()
    while True:
        
        param = {'offset' : page*20}
        data = requests.get(propub_url,headers=header,params=param).json()

        sponsor_id = data['results'][0]['id']
        sponsor_name = data['results'][0]['name']

        res.extend(data['results'][0]['bills']) # res
        
        print('page:',page)
        print('bills in page:',len(data['results'][0]['bills']))
        print('offset:',data['results'][0]['offset'])

        if len(data['results'][0]['bills']) < 20:
            print('### Done ###')
            clear()
            break
        page +=1
        if page%3 == 0:
            clear()
        print('===')

    
    if len(res) == 0:
        return {
            'sponsor_id':sponsor_id,
            'sponsor_name':sponsor_name,
            'Message':['No Bills '+bill_type.title()]
        }
    elif len(res) > 0:

        if get_subjects:
            res = add_bill_subjects(res)
        return res


In [21]:
hillary # Hillary Clinton (many bills)
smith = 'S001203' # Tina Smith (some bills)
lamb = 'L000588' # Connor Lamb (no bills)

In [23]:
#bills = get_bills(hillary)
#pd.DataFrame(bills).tail()

## Getting Votes
votes: https://projects.propublica.org/api-docs/congress-api/votes/#votes

In [103]:
import requests
import time
from IPython.display import clear_output as clear

'''Scrapes for all votes from most recent'''

chamber = 'both'

PROPUBLICA_API_KEY = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'

'''propub_url = ('https://api.propublica.org/congress/v1/{chamber}/votes/{start_date}/{end_date}.json'
              .format(chamber,
                  start_date,
                 end_date))'''

propub_url = 'https://api.propublica.org/congress/v1/{chamber}/votes/recent.json'.format(chamber=chamber)

header = {'X-API-Key' : PROPUBLICA_API_KEY}

stop = 3
res = list()
page = 0

while page < stop:
    param = {'offset' : page*20}
    data = requests.get(propub_url,headers=header,params=param).json()

    res.extend(data['results']['votes'])
    
    print('page:',page)
    page+=1
    
    if page&10 == 0:
        clear()
print('### Done! ###')

page: 1
page: 2
### Done! ###


In [120]:
res[0]

{'amendment': {},
 'bill': {},
 'chamber': 'Senate',
 'congress': 115,
 'date': '2018-04-19',
 'democratic': {'majority_position': 'No',
  'no': 47,
  'not_voting': 0,
  'present': 0,
  'yes': 0},
 'description': 'James Bridenstine, of Oklahoma, to be Administrator of the National Aeronautics and Space Administration',
 'independent': {'no': 2, 'not_voting': 0, 'present': 0, 'yes': 0},
 'nomination': {'agency': 'National Aeronautics and Space Administration',
  'name': 'James Bridenstine',
  'nomination_id': 'PN1359-115',
  'number': 'PN1359'},
 'question': 'On the Nomination',
 'republican': {'majority_position': 'Yes',
  'no': 0,
  'not_voting': 1,
  'present': 0,
  'yes': 50},
 'result': 'Nomination Confirmed',
 'roll_call': 80,
 'session': 2,
 'source': 'https://www.senate.gov/legislative/LIS/roll_call_votes/vote1152/vote_115_2_00080.xml',
 'time': '13:46:00',
 'total': {'no': 49, 'not_voting': 1, 'present': 0, 'yes': 50},
 'url': 'https://www.senate.gov/legislative/LIS/roll_call_l

In [174]:
res[6]

{'amendment': {},
 'bill': {'api_uri': 'https://api.propublica.org/congress/v1/115/bills/hres831.json',
  'bill_id': 'hres831-115',
  'latest_action': 'Motion to reconsider laid on the table Agreed to without objection.',
  'number': 'H.RES.831',
  'sponsor_id': 'N000189',
  'title': 'Providing for consideration of the bill (H.R. 5444) to amend the Internal Revenue Code of 1986 to modernize and improve the Internal Revenue Service, and for other purposes, and providing for consideration of the bill (H.R. 5445) to amend the Internal Revenue Code of 1986 to improve cybersecurity and taxpayer identity protection, and modernize the information technology of the Internal Revenue Service, and for other purposes.'},
 'chamber': 'House',
 'congress': 115,
 'date': '2018-04-18',
 'democratic': {'majority_position': 'No',
  'no': 188,
  'not_voting': 5,
  'present': 0,
  'yes': 0},
 'description': 'Providing for consideration of the bill (H.R. 5444) Taxpayer First Act, and providing for consider

In [182]:
res[6].keys()

dict_keys(['congress', 'chamber', 'session', 'roll_call', 'source', 'url', 'vote_uri', 'bill', 'amendment', 'question', 'description', 'vote_type', 'date', 'time', 'result', 'democratic', 'republican', 'independent', 'total'])

In [191]:
i = 1
vote = res[i]
roll_call = requests.get(vote['vote_uri'],headers=header).json()['results']['votes']['vote']
print(vote['question']+':',vote['description'])

for member in roll_call['positions']:
    member['question'] = vote['question']
    member['description'] = vote['description']
    member['date'] = vote['date']
    member['chamber'] = vote['chamber']

bill = vote['bill']
if len(bill) > 0:
    print('='*3)
    print('Bill:',vote['bill']['title'])
    print('Bill id:',vote['bill']['bill_id'])
    for member in roll_call['positions']:
        member['bill_id'],member['congress'] = bill['bill_id'].split('-')
        member['bill_sponsor'] = bill['sponsor_id']
else:
    for member in roll_call['positions']:
        member['bill_id'],member['congress'] = None,vote['congress']
        member['sponsor_id'] = None
pd.DataFrame(roll_call['positions']).tail()

On Motion to Suspend the Rules and Pass, as Amended: Justice for Victims of IRS Scams and Identity Theft Act of 2018
===
Bill: To require the Attorney General to establish procedures for expedited review of the case of any person who unlawfully solicits personal information for purposes of committing identity theft, while purporting to be acting on behalf of the IRS, and for other purposes.
Bill id: hr2905-115


,bill_id,bill_sponsor,chamber,congress,date,description,district,dw_nominate,member_id,name,party,question,state,vote_position
425,hr2905,Y000066,House,115,2018-04-18,Justice for Victims of IRS Scams and Identity ...,3,0.702,Y000065,Ted Yoho,R,"On Motion to Suspend the Rules and Pass, as Am...",FL,Yes
426,hr2905,Y000066,House,115,2018-04-18,Justice for Victims of IRS Scams and Identity ...,1,0.278,Y000033,Don Young,R,"On Motion to Suspend the Rules and Pass, as Am...",AK,Yes
427,hr2905,Y000066,House,115,2018-04-18,Justice for Victims of IRS Scams and Identity ...,3,0.448,Y000066,David Young,R,"On Motion to Suspend the Rules and Pass, as Am...",IA,Yes
428,hr2905,Y000066,House,115,2018-04-18,Justice for Victims of IRS Scams and Identity ...,1,0.355,Z000017,Lee Zeldin,R,"On Motion to Suspend the Rules and Pass, as Am...",NY,Yes
429,hr2905,Y000066,House,115,2018-04-18,Justice for Victims of IRS Scams and Identity ...,1,0.556,R000570,Paul D. Ryan,R,"On Motion to Suspend the Rules and Pass, as Am...",WI,Speaker


In [160]:
pd.DataFrame(res).iloc[1]['bill'].keys()

dict_keys(['bill_id', 'number', 'sponsor_id', 'api_uri', 'title', 'latest_action'])

In [165]:
from lxml import html
import requests
import time
from congress import Congress

propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'
congress = Congress(propublica_key)

def scrape_house_vote(vote_url):
    
    '''Scraps house vote data from https://clerk.house.gov/ url'''

    response = requests.get(vote_url)
    doc = html.fromstring(response.content)

    descrp = doc.xpath('//vote-desc')[0].text_content()
    legis_num = doc.xpath('//legis-num')[0].text_content().replace(' ','').replace('.','').lower()
    doc_congress = int(doc.xpath('//congress')[0].text_content())
    action_date = pd.to_datetime(doc.xpath('//action-date')[0].text_content())

    rep_id = [record.xpath('./legislator')[0].get('name-id') for record in doc.xpath('//vote-data/recorded-vote')]
    party = [record.xpath('./legislator')[0].get('party') for record in doc.xpath('//vote-data/recorded-vote')]
    state = [record.xpath('./legislator')[0].get('state') for record in doc.xpath('//vote-data/recorded-vote')]
    name = [record.xpath('./legislator')[0].text_content() for record in doc.xpath('//vote-data/recorded-vote')]
    vote = [record.xpath('./vote')[0].text_content() for record in doc.xpath('//vote-data/recorded-vote')]

    res = {
        'bill_id': legis_num,
        'date' : action_date,
        'congress' : doc_congress,
        'vote' : vote,
        'bioguide' : rep_id,
        'lis_id' : None,
        'member_name' : name,
        'member_chamber' : 'house',
        'party' : party,
        'state' : state,
        'description' : descrp
        }

    return res

def scrape_senate_vote(vote_url):
    
    '''Scraps senate vote data from https://www.senate.gov/ url'''

    response = requests.get(vote_url)
    doc = html.fromstring(response.content)

    vote_title = doc.xpath('//vote_title')[0].text_content()
    legis_num = doc.xpath('//document_number')[0].text_content()
    doc_name = doc.xpath('//document_name')[0].text_content().replace('.','').replace(' ','').lower()
    descrp = doc.xpath('//document_title')[0].text_content()
    doc_congress = int(doc.xpath('//document_congress')[0].text_content())
    action_date = pd.to_datetime(doc.xpath('//vote_date')[0].text_content()[:11])


    name = [member.xpath('./member_full')[0].text_content() for member in doc.xpath('//member')]
    first_name = [member.xpath('./first_name')[0].text_content() for member in doc.xpath('//member')]
    vote = [member.xpath('./vote_cast')[0].text_content() for member in doc.xpath('//member')]
    lis_id = [member.xpath('./lis_member_id')[0].text_content() for member in doc.xpath('//member')]
    party = [member.xpath('./party')[0].text_content() for member in doc.xpath('//member')]
    state = [member.xpath('./state')[0].text_content() for member in doc.xpath('//member')]

    res = {
        'bill_id': legis_num,
        'date' : action_date,
        'congress' : doc_congress,
        'vote' : vote,
        'bioguide': None,
        'lis_id' : lis_id,
        'member_name' : name,
        'member_chamber':'senate',
        'party' : party,
        'state' : state,
        'description' : descrp,
        }

    return res

def scrape_vote(vote_url):
    
    '''Scrapes vote from url'''
    
    if 'clerk' in vote_url:
        return scrape_house_vote(vote_url)
    elif 'senate' in vote_url:
        return scrape_senate_vote(vote_url)

import datetime as dt
import pandas as pd
from IPython.display import clear_output as clear
import os

def get_votes(chamber, folder_name = 'Data', file_name = None,
              start_year=dt.datetime.now().year,
              end_year=dt.datetime.now().year):
    
    '''Uploads vote data to json file'''
    
    t0 = time.time()
    
    # json_path
    if file_name == None:
        file_name = chamber+'_votes.json'
    elif '.json' not in file_name:
        file_name = file_name+'.json'
    file_name = '/' + file_name
    folder_name = '/' + folder_name
    cwd = os.getcwd()
    json_path = cwd+folder_name+file_name
    
    if not os.path.isdir(cwd+folder_name):
        os.mkdir(cwd+folder_name)
        print('* New dir made: ',os.getcwd()+folder_name)

    # getting urls
    vote_list = list()
    for year in range(start_year,end_year+1):
        print('=== Getting urls ===')
        print('start:',start_year)
        print('end:',end_year)
        print('='*3)
        for month in range(0,12):
            votes = congress.votes.by_month(chamber,year=year,month=month)['votes']
            if len(votes) == 0:
                print(year,month,'passed')
                pass
            else:
                votes_urls = [vote['source'] for vote in votes]
                vote_list.extend(vote_urls)
                print(year,month)
        clear()
    vote_list.reverse()
    
    # uploading to pandas
    vote_data = pd.DataFrame(scrape_vote(vote_list[0]))
    cols = vote_data.columns
    vote_data[cols].to_json(json_path)
    
    data_len = len(vote_list)
    
    for i in range(1,data_len):
        
        vote_data = pd.read_json(json_path)
        print('='*5)
        print('Old data length:', len(vote_data))
        print('Getting dataframe:', data_len-i, 'left')
        
        vote_data.append(pd.DataFrame(scrape_vote(vote_list[i])))
        vote_data[cols].to_json(json_path)
        print('New data length:', len(vote_data))
        
        if i%10 == 0:
            clear()
            timer = time.time()-t0
            print('Time passed:',
                  int(timer/60),'m',
                  round(timer%60,2),'s')
        
    
    timer = time.time()-t0
    print('#'*3,'DONE','#'*3)
    print('Time:',int(timer),'m',round(timer%60,2),'s')
    print('path:',json_path)

In [166]:
file_name = None
folder_name = 'Data'

if file_name == None:
    file_name = chamber+'_votes.json'
elif '.json' not in file_name:
    file_name = file_name+'.json'

file_name = '/' + file_name
folder_name = '/' + folder_name

json_path = os.getcwd()+folder_name+file_name
json_path

'/Users/Xie/Documents/School/Senior Spring/PPDS/Data/house_votes.json'

In [167]:
chamber = 'house'
vote = congress.votes.by_month(chamber,year=2018,month=3)['votes'][0]
pd.DataFrame(scrape_vote(vote['source'])).tail(3)

,bill_id,bioguide,congress,date,description,lis_id,member_chamber,member_name,party,state,vote
426,hr5089,Y000033,115,2018-03-22,Strengthening Local Transportation Security Ca...,None,house,Young (AK),R,AK,Yea
427,hr5089,Y000066,115,2018-03-22,Strengthening Local Transportation Security Ca...,None,house,Young (IA),R,IA,Yea
428,hr5089,Z000017,115,2018-03-22,Strengthening Local Transportation Security Ca...,None,house,Zeldin,R,NY,Yea


In [ ]:
vote_data.head()

In [ ]:
for i in range(0,3):
    year = 2018-i
    print('='*3,year,'='*3)
    for month in range(1,13):
        print(month,'house:',len(congress.votes.by_month('house',year=year,month=month)['votes']))
        print(month,'senate:',len(congress.votes.by_month('senate',year=year,month=month)['votes']))
        print('-'*3)
print('### end ###')

In [ ]:
scrape_vote(vote_url,chamber)

In [ ]:
chamber = 'senate'
votes = congress.votes.by_month(chamber)['votes']
for i in range(len(votes)):
    vote_url = votes[i]['source']
    pd.DataFrame(scrape_vote(vote_url,chamber))

In [ ]:
# compare
congress.members.compare('S000033','C001098','senate',congress=cong)

In [ ]:
# passed bills
pd.DataFrame(congress.bills.passed('senate')['bills']).tail(3)